 # Lab 4: 품질 평가 - 용어 일관성



 용어집 준수 여부와 브랜드명 정확도를 평가합니다.



 ## 학습 목표



 - 용어집 준수 여부 자동 평가

 - 브랜드명/제품명 정확도 검증

 - 용어 일관성이 사용자 경험에 미치는 영향 이해



 ## 워크샵 전체 흐름



 ```

 Lab 1: 환경 설정 ✅

 ↓

 Lab 2: 번역 기법 비교 ✅

 ↓

 Lab 3: 품질 평가 - 충실도 ✅

 ↓

 Lab 4: 품질 평가 - 용어 일관성 (현재)

 ↓

 Lab 5: 품질 평가 - 문화/톤

 ↓

 Lab 6: 피드백 기반 재번역

 ↓

 Lab 7: 종합 분석

 ```

 ---

 ## 4.1 환경 설정 및 이전 Lab 결과 로드

In [1]:
import json
import os
from datetime import datetime

import boto3
import pandas as pd


In [2]:
# 설정
REGION = "us-west-2"
OUTPUT_DIR = "lab_outputs"

bedrock_client = boto3.client(
    service_name="bedrock-runtime",
    region_name=REGION
)

CLAUDE_MODELS = {
    "sonnet": {
        "id": "us.anthropic.claude-sonnet-4-20250514-v1:0",
        "name": "Claude Sonnet 4",
    },
    "haiku": {
        "id": "us.anthropic.claude-haiku-4-20250514-v1:0",
        "name": "Claude Haiku 4",
    }
}

DEFAULT_MODEL = "sonnet"
MODEL_ID = CLAUDE_MODELS[DEFAULT_MODEL]["id"]

print(f"Bedrock 클라이언트 초기화 완료")
print(f"사용 모델: {CLAUDE_MODELS[DEFAULT_MODEL]['name']}")


Bedrock 클라이언트 초기화 완료
사용 모델: Claude Sonnet 4


In [3]:
# 유틸리티 함수
def load_results(filename: str) -> dict:
    filepath = os.path.join(OUTPUT_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        data = json.load(f)
    print(f"✓ 로드 완료: {filepath}")
    return data

def save_results(data: dict, filename: str) -> str:
    filepath = os.path.join(OUTPUT_DIR, filename)
    data["_metadata"] = {
        "saved_at": datetime.now().isoformat(),
        "model": CLAUDE_MODELS[DEFAULT_MODEL]["name"]
    }
    with open(filepath, "w", encoding="utf-8") as f:
        json.dump(data, f, ensure_ascii=False, indent=2)
    print(f"✓ 저장 완료: {filepath}")
    return filepath

def call_claude(prompt: str, model_id: str = MODEL_ID, max_tokens: int = 2048) -> str:
    request_body = {
        "anthropic_version": "bedrock-2023-05-31",
        "max_tokens": max_tokens,
        "messages": [{"role": "user", "content": prompt}]
    }
    response = bedrock_client.invoke_model(
        modelId=model_id,
        body=json.dumps(request_body),
        contentType="application/json",
        accept="application/json"
    )
    response_body = json.loads(response["body"].read())
    return response_body["content"][0]["text"]


In [4]:
# Lab 2 결과 로드
zero_shot_data = load_results("translations_zero_shot.json")
few_shot_data = load_results("translations_few_shot.json")
glossary_data = load_results("translations_glossary.json")

print(f"\n로드된 번역 결과:")
print(f"  - Zero-shot: {len(zero_shot_data['translations'])}건")
print(f"  - Few-shot: {len(few_shot_data['translations'])}건")
print(f"  - 용어집: {len(glossary_data['translations'])}건")


✓ 로드 완료: lab_outputs/translations_zero_shot.json
✓ 로드 완료: lab_outputs/translations_few_shot.json
✓ 로드 완료: lab_outputs/translations_glossary.json

로드된 번역 결과:
  - Zero-shot: 6건
  - Few-shot: 6건
  - 용어집: 6건


 ---

 ## 4.2 용어 일관성 평가란?



 용어 일관성 평가는 번역이 정해진 용어집을 얼마나 잘 따르는지 측정합니다.



 ### 평가 항목

 - 브랜드명 보존: Samsung Cloud, Samsung account 등 번역하면 안 되는 용어

 - 기술 용어 일관성: 동기화/sync, 백업/backup 등 일관된 번역

 - 제품명 정확도: Galaxy, OneDrive 등 고유명사 처리



 ### 점수 체계 (0-5)



 | 점수 | 판정 | 설명 |

 |------|------|------|

 | 5 | Pass | 모든 용어 정확, 일관성 완벽 |

 | 4 | Pass | 1건 경미한 불일치 |

 | 3 | Review | 다수 용어 미매핑 (검수 필요) |

 | 0-2 | Fail | 브랜드/법적 명칭 오기 |

 ---

 ## 4.3 삼성클라우드 용어집 정의

In [5]:
# 삼성클라우드 용어집 (Lab 2에서 사용한 것과 동일)
GLOSSARY = {
    # 브랜드명 (번역 금지)
    "Samsung Cloud": "Samsung Cloud",
    "Samsung account": "Samsung account",
    "Galaxy": "Galaxy",
    "OneDrive": "OneDrive",
    
    # 기술 용어 (일관된 번역)
    "동기화": "sync",
    "백업": "backup",
    "복원": "restore",
    "단말": "device",
    "저장공간": "storage",
    "휴지통": "Trash",  # US 스타일
    "설정": "Settings",
    "계정": "account",
}

print("삼성클라우드 용어집:")
print("=" * 50)
print("\n[브랜드명 - 번역 금지]")
for ko, en in list(GLOSSARY.items())[:4]:
    print(f"  {ko} → {en}")

print("\n[기술 용어 - 일관된 번역]")
for ko, en in list(GLOSSARY.items())[4:]:
    print(f"  {ko} → {en}")


삼성클라우드 용어집:

[브랜드명 - 번역 금지]
  Samsung Cloud → Samsung Cloud
  Samsung account → Samsung account
  Galaxy → Galaxy
  OneDrive → OneDrive

[기술 용어 - 일관된 번역]
  동기화 → sync
  백업 → backup
  복원 → restore
  단말 → device
  저장공간 → storage
  휴지통 → Trash
  설정 → Settings
  계정 → account


 ---

 ## 4.4 용어 일관성 평가 프롬프트 설계

In [6]:
TERMINOLOGY_EVAL_PROMPT = """당신은 번역 품질 평가 전문가입니다.
아래 원문, 번역문, 용어집을 비교하여 용어 일관성을 평가해주세요.

## 원문 (한국어)
{source}

## 번역문 (영어)
{translation}

## 용어집
{glossary_text}

## 평가 기준
- 브랜드명 보존: Samsung Cloud, Samsung account, Galaxy 등이 번역되지 않고 그대로 유지되었는가?
- 기술 용어 일관성: 용어집에 정의된 대로 번역되었는가?
- 용어 누락: 용어집에 있는 단어가 다르게 번역되었는가?

## 점수 기준
- 5점: 모든 용어 정확, 일관성 완벽
- 4점: 1건 경미한 불일치
- 3점: 다수 용어 미매핑 (검수 필요)
- 0-2점: 브랜드/법적 명칭 오기

## 출력 형식
반드시 아래 JSON 형식으로만 응답하세요:
```json
{{
  "score": <0-5 사이 정수>,
  "verdict": "<Pass|Review|Fail>",
  "term_checks": [
 {{"term": "용어", "expected": "기대값", "found": "실제값", "correct": true/false}}
  ],
  "issues": ["발견된 문제점"],
  "suggestion": "개선 제안",
  "reasoning": "평가 근거를 한국어로 설명"
}}
```"""


 ---

 ## 4.5 평가 함수 정의

In [7]:
def evaluate_terminology(source: str, translation: str, glossary: dict) -> dict:
    """용어 일관성 평가를 수행하고 결과를 반환합니다."""
    
    glossary_text = "\n".join([f"- {ko} → {en}" for ko, en in glossary.items()])
    
    prompt = TERMINOLOGY_EVAL_PROMPT.format(
        source=source,
        translation=translation,
        glossary_text=glossary_text
    )
    
    response = call_claude(prompt)
    
    try:
        if "```json" in response:
            json_str = response.split("```json")[1].split("```")[0].strip()
        elif "```" in response:
            json_str = response.split("```")[1].split("```")[0].strip()
        else:
            json_str = response.strip()
        result = json.loads(json_str)
    except (json.JSONDecodeError, IndexError) as e:
        result = {"score": 0, "verdict": "Fail", "term_checks": [], "issues": ["파싱 실패"], "suggestion": "", "reasoning": str(e)}
    
    return result


In [8]:
# 평가 테스트
test_source = "삼성 클라우드에서 동기화된 데이터를 확인하세요"
test_translation = "Check your synced data in Samsung Cloud"

print("평가 테스트")
print(f"원문: {test_source}")
print(f"번역: {test_translation}")
print("-" * 50)

test_result = evaluate_terminology(test_source, test_translation, GLOSSARY)
print(f"점수: {test_result['score']}")
print(f"판정: {test_result['verdict']}")
print(f"근거: {test_result['reasoning']}")


평가 테스트
원문: 삼성 클라우드에서 동기화된 데이터를 확인하세요
번역: Check your synced data in Samsung Cloud
--------------------------------------------------
점수: 5
판정: Pass
근거: 브랜드명 'Samsung Cloud'가 정확히 보존되었고, '동기화'가 용어집 기준에 따라 'sync'의 과거분사형 'synced'로 적절히 번역되었습니다. 문법적으로도 자연스럽고 용어 일관성이 완벽합니다.


 ---

 ## 4.6 세 가지 번역 기법 평가 실행

In [9]:
def run_terminology_evaluation(translations: list, method_name: str) -> list:
    """번역 목록에 대해 용어 일관성 평가를 실행합니다."""
    
    print(f"\n{method_name} 용어 일관성 평가 중...")
    print("=" * 80)
    
    results = []
    
    for item in translations:
        eval_result = evaluate_terminology(item["source"], item["translation"], GLOSSARY)
        
        result = {
            "id": item["id"],
            "source": item["source"],
            "translation": item["translation"],
            "method": item["method"],
            "score": eval_result["score"],
            "verdict": eval_result["verdict"],
            "term_checks": eval_result.get("term_checks", []),
            "issues": eval_result["issues"],
            "suggestion": eval_result["suggestion"],
            "reasoning": eval_result["reasoning"],
            "eval_type": "terminology",
            "timestamp": datetime.now().isoformat()
        }
        results.append(result)
        
        status = "✓" if eval_result["verdict"] == "Pass" else "△" if eval_result["verdict"] == "Review" else "✗"
        print(f"\n[{item['id']}] {status} 점수: {eval_result['score']}/5")
        print(f"  번역: {item['translation'][:50]}...")
        if eval_result["issues"]:
            print(f"  문제: {', '.join(eval_result['issues'][:2])}")
    
    return results


In [10]:
# Zero-shot 평가
zero_shot_eval = run_terminology_evaluation(
    zero_shot_data["translations"], 
    "Zero-shot"
)



Zero-shot 용어 일관성 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  번역: Do I need a Samsung account to use Samsung Cloud?...

[IDS_FAQ_SC_MAIN_HEADER_03] ✓ 점수: 5/5
  번역: What happens when you remove a Samsung account fro...

[IDS_FAQ_SC_ACCESSING_TEXT] ✓ 점수: 5/5
  번역: After accessing the Samsung Cloud webpage, please ...

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 5/5
  번역: I don't currently have a Galaxy device. Is there a...

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 5/5
  번역: Support availability may vary depending on the spe...

[IDS_FAQ_GO_MAIN_HEADER_27] ✗ 점수: 0/5
  번역: Where can I go to check the cloud recycle bin?...
  문제: 용어집에서 '휴지통'은 'Trash'로 번역되어야 하나 'recycle bin'으로 번역됨


In [11]:
# Few-shot 평가
few_shot_eval = run_terminology_evaluation(
    few_shot_data["translations"], 
    "Few-shot"
)



Few-shot 용어 일관성 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  번역: Do I need a Samsung account to use Samsung Cloud?...

[IDS_FAQ_SC_MAIN_HEADER_03] ✓ 점수: 5/5
  번역: What happens if I remove my Samsung account from m...

[IDS_FAQ_SC_ACCESSING_TEXT] ✓ 점수: 5/5
  번역: After accessing the Samsung Cloud webpage, you can...

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 4/5
  번역: I don't currently have a Galaxy device. Is there a...
  문제: '동기화'가 용어집 기준 'sync'가 아닌 'synchronized'로 번역됨

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 5/5
  번역: Support may vary depending on the specific device,...

[IDS_FAQ_GO_MAIN_HEADER_27] ✓ 점수: 5/5
  번역: Where can I go to check my Cloud Trash?...


In [12]:
# 용어집 적용 평가
glossary_eval = run_terminology_evaluation(
    glossary_data["translations"], 
    "용어집 적용"
)



용어집 적용 용어 일관성 평가 중...

[IDS_FAQ_SC_MAIN_HEADER_02] ✓ 점수: 5/5
  번역: Do I need a Samsung account to use Samsung Cloud?...

[IDS_FAQ_SC_MAIN_HEADER_03] ✓ 점수: 5/5
  번역: What happens when you remove your Samsung account ...

[IDS_FAQ_SC_ACCESSING_TEXT] ✓ 점수: 5/5
  번역: After accessing the Samsung Cloud webpage, please ...

[IDS_FAQ_SC_MAIN_HEADER_13] ✓ 점수: 5/5
  번역: I don't currently have a Galaxy device. Is there a...

[IDS_FAQ_GO_DEVICE_NOTES_UL_01] ✓ 점수: 5/5
  번역: Support may vary depending on the specific device,...

[IDS_FAQ_GO_MAIN_HEADER_27] ✓ 점수: 5/5
  번역: Where can I go to check the cloud Trash?...


 ---

 ## 4.7 용어 위반 사례 분석



 각 기법별로 용어집을 얼마나 잘 따랐는지 비교합니다.

In [13]:
print("=" * 80)
print("용어 위반 사례 분석")
print("=" * 80)

# 휴지통 번역 비교 (Trash vs Recycle bin)
print("\n[휴지통 번역 비교]")
print("용어집 정의: 휴지통 → Trash (US 스타일)")
print("-" * 50)

for i, item in enumerate(zero_shot_data["translations"]):
    if "휴지통" in item["source"]:
        print(f"\n원문: {item['source']}")
        print(f"Zero-shot: {zero_shot_data['translations'][i]['translation']}")
        print(f"Few-shot:  {few_shot_data['translations'][i]['translation']}")
        print(f"용어집:    {glossary_data['translations'][i]['translation']}")


용어 위반 사례 분석

[휴지통 번역 비교]
용어집 정의: 휴지통 → Trash (US 스타일)
--------------------------------------------------

원문: 어디로 가면 클라우드 휴지통을 확인할 수 있나요?
Zero-shot: Where can I go to check the cloud recycle bin?
Few-shot:  Where can I go to check my Cloud Trash?
용어집:    Where can I go to check the cloud Trash?


 ---

 ## 4.8 평가 결과 비교

In [14]:
def summarize_scores(eval_results: list, method_name: str) -> dict:
    scores = [r["score"] for r in eval_results]
    verdicts = [r["verdict"] for r in eval_results]
    
    return {
        "method": method_name,
        "avg_score": sum(scores) / len(scores),
        "pass_count": verdicts.count("Pass"),
        "review_count": verdicts.count("Review"),
        "fail_count": verdicts.count("Fail"),
        "total": len(scores)
    }

zero_summary = summarize_scores(zero_shot_eval, "Zero-shot")
few_summary = summarize_scores(few_shot_eval, "Few-shot")
glossary_summary = summarize_scores(glossary_eval, "용어집")


In [15]:
print("=" * 80)
print("용어 일관성 평가 결과 비교")
print("=" * 80)

print(f"\n{'기법':<15} {'평균점수':<10} {'Pass':<8} {'Review':<8} {'Fail':<8}")
print("-" * 50)

for summary in [zero_summary, few_summary, glossary_summary]:
    print(f"{summary['method']:<15} {summary['avg_score']:<10.2f} {summary['pass_count']:<8} {summary['review_count']:<8} {summary['fail_count']:<8}")

print("\n" + "=" * 80)


용어 일관성 평가 결과 비교

기법              평균점수       Pass     Review   Fail    
--------------------------------------------------
Zero-shot       4.17       5        0        1       
Few-shot        4.83       6        0        0       
용어집             5.00       6        0        0       



In [16]:
# 항목별 상세 비교
print("\n항목별 점수 비교")
print("=" * 80)
print(f"{'ID':<35} {'Zero':<8} {'Few':<8} {'Glossary':<8}")
print("-" * 60)

for i in range(len(zero_shot_eval)):
    item_id = zero_shot_eval[i]["id"]
    zero_score = zero_shot_eval[i]["score"]
    few_score = few_shot_eval[i]["score"]
    glossary_score = glossary_eval[i]["score"]
    
    print(f"{item_id:<35} {zero_score:<8} {few_score:<8} {glossary_score:<8}")

print("=" * 80)



항목별 점수 비교
ID                                  Zero     Few      Glossary
------------------------------------------------------------
IDS_FAQ_SC_MAIN_HEADER_02           5        5        5       
IDS_FAQ_SC_MAIN_HEADER_03           5        5        5       
IDS_FAQ_SC_ACCESSING_TEXT           5        5        5       
IDS_FAQ_SC_MAIN_HEADER_13           5        4        5       
IDS_FAQ_GO_DEVICE_NOTES_UL_01       5        5        5       
IDS_FAQ_GO_MAIN_HEADER_27           0        5        5       


 ---

 ## 4.9 충실도 vs 용어 일관성 비교



 Lab 3(충실도)와 Lab 4(용어) 결과를 비교하여 트레이드오프를 확인합니다.

In [17]:
# Lab 3 결과 로드
faithfulness_data = load_results("eval_faithfulness.json")

print("=" * 80)
print("충실도 vs 용어 일관성 비교")
print("=" * 80)

print(f"\n{'기법':<15} {'충실도':<12} {'용어 일관성':<12} {'차이':<10}")
print("-" * 50)

faith_summaries = faithfulness_data["summary"]
term_summaries = {"zero_shot": zero_summary, "few_shot": few_summary, "glossary": glossary_summary}

for key, name in [("zero_shot", "Zero-shot"), ("few_shot", "Few-shot"), ("glossary", "용어집")]:
    faith_avg = faith_summaries[key]["avg_score"]
    term_avg = term_summaries[key]["avg_score"]
    diff = term_avg - faith_avg
    
    print(f"{name:<15} {faith_avg:<12.2f} {term_avg:<12.2f} {diff:+.2f}")

print("\n" + "-" * 50)
print("해석:")
print("  - 용어집 적용 시 용어 일관성 점수가 높아짐")
print("  - Few-shot도 예시를 통해 용어 스타일을 학습")
print("=" * 80)


✓ 로드 완료: lab_outputs/eval_faithfulness.json
충실도 vs 용어 일관성 비교

기법              충실도          용어 일관성       차이        
--------------------------------------------------
Zero-shot       4.83         4.17         -0.67
Few-shot        4.50         4.83         +0.33
용어집             4.50         5.00         +0.50

--------------------------------------------------
해석:
  - 용어집 적용 시 용어 일관성 점수가 높아짐
  - Few-shot도 예시를 통해 용어 스타일을 학습


 ---

 ## 4.10 평가 결과 저장

In [18]:
terminology_results = {
    "zero_shot": zero_shot_eval,
    "few_shot": few_shot_eval,
    "glossary": glossary_eval,
    "summary": {
        "zero_shot": zero_summary,
        "few_shot": few_summary,
        "glossary": glossary_summary
    }
}

save_results(terminology_results, "eval_terminology.json")


✓ 저장 완료: lab_outputs/eval_terminology.json


'lab_outputs/eval_terminology.json'

 ---

 ## 4.11 Lab 4 완료

In [19]:
print("=" * 80)
print("Lab 4: 품질 평가 - 용어 일관성 완료")
print("=" * 80)

print("\n[평가 결과 요약]")
for summary in [zero_summary, few_summary, glossary_summary]:
    print(f"  - {summary['method']}: 평균 {summary['avg_score']:.2f}점, Pass {summary['pass_count']}건")

print("\n[핵심 인사이트]")
print("  - 용어집 적용 번역이 용어 일관성에서 가장 높은 점수")
print("  - Few-shot도 예시를 통해 용어 스타일 학습 가능")
print("  - 충실도와 용어 일관성은 별개의 평가 축")

print("\n[저장된 파일]")
print(f"  - {OUTPUT_DIR}/eval_terminology.json")

print("\n[다음 단계]")
print("  → Lab 5: 품질 평가 - 문화/톤")
print("    존대/격식, 브랜드 보이스 일관성을 평가합니다.")

print("\n" + "=" * 80)


Lab 4: 품질 평가 - 용어 일관성 완료

[평가 결과 요약]
  - Zero-shot: 평균 4.17점, Pass 5건
  - Few-shot: 평균 4.83점, Pass 6건
  - 용어집: 평균 5.00점, Pass 6건

[핵심 인사이트]
  - 용어집 적용 번역이 용어 일관성에서 가장 높은 점수
  - Few-shot도 예시를 통해 용어 스타일 학습 가능
  - 충실도와 용어 일관성은 별개의 평가 축

[저장된 파일]
  - lab_outputs/eval_terminology.json

[다음 단계]
  → Lab 5: 품질 평가 - 문화/톤
    존대/격식, 브랜드 보이스 일관성을 평가합니다.

